>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=String_Tracking)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=String_Tracking) to leverage the power of whylogs and WhyLabs together!*

# String Tracking - Unicode Range and String Length

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/advanced/String_Tracking.ipynb)

By default, columns of type `str` will have the following metrics, when logged with whylogs:
- Counts
- Types
- Frequent Items/Frequent Strings
- Cardinality

In this example, we'll see how you can track further metrics for string columns. We will do that by counting, for each string record, the number of characters that fall in a given unicode range, and then generating distribution metrics, such as `mean`, `stddev` and quantile values based on these counts. In addition to specific unicode ranges, we'll do the same approach, but for the overall string length.

In this example, we're interested in tracking two specific ranges of characters:
- ASCII Digits (unicode range 48-57)
- Latin alphabet (unicode range 97-122)

For more info on the unicode list of characters, check this [Wikipedia Article](https://en.wikipedia.org/wiki/List_of_Unicode_characters)

## Installing whylogs

If you haven't already, install whylogs: 

In [1]:
%pip install whylogs

## Creating the Data

Let's create a simple dataframe to demonstrate. To better visualize how the metrics work, we'll create 3 columns:
- `onlyDigits`: Column of strings that contain only digit characters
- `onlyAlpha`: Column of strings that contain only latin letters (no digits)
- `mixed`: Column of strings that contain, digits, letters and other types of charachters, like punctuation and symbols

In [3]:
import whylogs as why
import pandas as pd
data = {
    "onlyDigits": ["12", "83", "1", "992", "7"],
    "onlyAlpha": ["Alice", "Bob", "Chelsea", "Danny", "Eddie"],
    "mixed": ["my_email_1989@gmail.com","ADK-1171","Copacabana 272 - Rio de Janeiro","21º C Friday - Sao Paulo, Brasil","18127819ASW"]
}
df = pd.DataFrame(data)
df.head()

,onlyDigits,onlyAlpha,mixed
0,12,Alice,my_email_1989@gmail.com
1,83,Bob,ADK-1171
2,1,Chelsea,Copacabana 272 - Rio de Janeiro
3,992,Danny,"21º C Friday - Sao Paulo, Brasil"
4,7,Eddie,18127819ASW


## Configuring the Metrics in the DatasetSchema

whylogs uses `Resolvers` in order to define the set of metrics tracked for a column name or data type.
In this case, we'll create a custom Resolver to apply the UnicodeRangeMetric to all of the columns.

> If you're interested in seeing how you can add or remove different metrics according to the column type or column name, please refer to this example on [Schema Configuration](https://whylogs.readthedocs.io/en/stable/examples/basic/Schema_Configuration.html)


In [5]:
from whylogs.core.schema import ColumnSchema, DatasetSchema
from whylogs.core.metrics.unicode_range import UnicodeRangeMetric
from whylogs.core.resolvers import Resolver
from whylogs.core.datatypes import DataType
from typing import Dict
from whylogs.core.metrics import Metric, MetricConfig

class UnicodeResolver(Resolver):
    def resolve(self, name: str, why_type: DataType, column_schema: ColumnSchema) -> Dict[str, Metric]:
        return {UnicodeRangeMetric.get_namespace(column_schema.cfg): UnicodeRangeMetric.zero(column_schema.cfg)}

Resolvers are passed to whylogs through a `DatasetSchema`, so we'll have to create a custom Schema as well.

We'll just have to:
- Pass the UnicodeResolver created previously
- Since we're interested in changing the default character ranges, we'll also pass a Metric Configuration with the desired ranges

In [6]:
config = MetricConfig(unicode_ranges={"digits": (48, 57), "alpha": (97, 122)})
schema = DatasetSchema(resolvers=UnicodeResolver(), default_configs=config)


If a default MetricConfig is not passed, it would use the default unicode ranges, which would track the default ranges such as: emoticons, control characters and extended latin. 

We can now log the dataframe and pass our schema when calling `log`:

In [7]:
import whylogs as why

prof_results = why.log(df, schema=DatasetSchema(resolvers=UnicodeResolver(), default_configs=MetricConfig(unicode_ranges={"digits": (48, 57), "alpha": (97, 122)})))
prof = prof_results.profile()

## Unicode Range and String Length Metrics

Let's take a look at the __Profile View__:

In [8]:
profile_view_df = prof.view().to_pandas()
profile_view_df

,unicode_range/unicode_range/digits/mean,unicode_range/unicode_range/digits/stddev,unicode_range/unicode_range/digits/n,unicode_range/unicode_range/digits/max,unicode_range/unicode_range/digits/min,unicode_range/unicode_range/digits/q_01,unicode_range/unicode_range/digits/q_05,unicode_range/unicode_range/digits/q_10,unicode_range/unicode_range/digits/q_25,unicode_range/unicode_range/digits/median,...,unicode_range/unicode_range/string_length/q_01,unicode_range/unicode_range/string_length/q_05,unicode_range/unicode_range/string_length/q_10,unicode_range/unicode_range/string_length/q_25,unicode_range/unicode_range/string_length/median,unicode_range/unicode_range/string_length/q_75,unicode_range/unicode_range/string_length/q_90,unicode_range/unicode_range/string_length/q_95,unicode_range/unicode_range/string_length/q_99,type
column,,,,,,,,,,,,,,,,,,,,,
onlyDigits,1.8,0.748331,5,3.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,3.0,SummaryType.COLUMN
mixed,4.2,2.039608,5,8.0,2.0,2.0,2.0,2.0,3.0,4.0,...,8.0,8.0,8.0,11.0,23.0,31.0,32.0,32.0,32.0,SummaryType.COLUMN
onlyAlpha,0.0,0.000000,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,5.0,5.0,5.0,7.0,7.0,7.0,SummaryType.COLUMN


You can see there's a lot of different metrics for each of the original dataframe's columns. In the `unicode_range` metric, we'll have additional sub metrics. In this case, we have metrics for:

- digits: distribution metrics for characters inside the unicode's digit range
- alpha: distribution metrics for characters inside the unicode's lowercase letters range
- UNKNOWN: distribution metrics for character that fall anywhere outside the predefined range (digits and alpha)
- string_length: distribution metrics for overall string length

For each of these submetrics, we have metric components such as:

- mean: the calculated mean for the column
- stddev: the calculated standard deviation for the column
- n: the total number of record for the column
- max, min: maximum and minimum values for the column
- q_xx: the xx-th quantile value of the data’s distribution
- median: the median for the column

For instance, let's check the mean for `alpha`

In [9]:
profile_view_df['unicode_range/unicode_range/alpha/mean']

column
onlyDigits     0.0
mixed         12.8
onlyAlpha      5.0
Name: unicode_range/unicode_range/alpha/mean, dtype: float64

The above values shows a mean of 0 for `onlyDigits` - which is expected, since we don't have any letters in this column, only digits. We also have a mean of 5 for `onlyAlpha`, which will coincide of the string's length mean for the same column, since we only have letters characters in this columns. For `mixed` the mean is 12.8, and we can indeed see that this column has a higher count of letter character than the previous columns.

> You might notice that, even though we defined the range for only lowercase letters, uppercase characters also are included when calculating the metrics. That happens because the strings are all lowercased during preprocessing before tracking the strings. 

Let's now check the `UNKNOWN` namespace: 

In [10]:
profile_view_df['unicode_range/unicode_range/UNKNOWN/mean']

column
onlyDigits     0.0
mixed         24.8
onlyAlpha      5.0
Name: unicode_range/unicode_range/UNKNOWN/mean, dtype: float64

Since we have only digits and letters in `onlyDigit` and `onlyAlpha`, there are no characters outside of the defined ranges, yielding means of 0. In the `mixed`, however, this value is non-zero, since there are characters such as `., -, º`, and whitespaces, that are not in any of the predefined ranges.

The last namespace `string_lenth`, contains metrics for the string's length:

In [11]:
profile_view_df['unicode_range/unicode_range/string_length/min']

column
onlyDigits    1.0
mixed         8.0
onlyAlpha     3.0
Name: unicode_range/unicode_range/string_length/min, dtype: float64

The `string_length` doesn't take into account any particular range. It containts aggregate metrics for the overall string length of each column. In this case, we're seeing the minimum value for the 3 columns: 1 for `onlyDigits`, 3 for `onlyAlpha` and 8 for `mixed`. Since the dataframe used here is very small, we can easily check the original data and verify that these metrics are indeed correct.

## Conclusion

Feel free to define your own ranges of interest and combine the UnicodeRange metrics with other standard metrics as you see fit!

The resulting profiles can be:
- merged together
- stored locally or in the cloud (AWS' S3)

or used for other purposes, such as:
- Setting constraints for data quality validation
- Visualizing and comparing profiles
- Sent to monitoring and observability platforms

Be sure to check the other examples at [whylogs' Documentation](https://whylogs.readthedocs.io/en/stable/examples.html)!